# Imports

In [ ]:
import os
import selenium.webdriver as webdriver
from selenium.webdriver.edge.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import datetime

# Parameters

In [ ]:
betfair_url_today = 'https://www.betfair.com/exchange/plus/pt/futebol-apostas-1/today'
betfair_url_tomorrow = 'https://www.betfair.com/exchange/plus/pt/futebol-apostas-1/tomorrow'

betfair_email = 'viniciusbonelli9@gmail.com' #FILL WITH YOUR CREDENTIALS
betfair_password = 'GTI$foy6c57' #FILL WITH YOUR CREDENTIALS

base_xpath = '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[3]/section'

# Functions

# Main Code

### Opening Betfair

In [ ]:
#GET EDGE WEBDRIVER
edge_driver_path = os.path.join(os.getcwd(), 'msedgedriver.exe')
edge_service = Service(edge_driver_path)

#OPEN BROWSER SESSION
browser = webdriver.Edge(service = edge_service)
browser.get(betfair_url_tomorrow) #CHANGE HERE BASED ON WHICH DAY YOU WANT TO COLLECT DATA FROM
browser.maximize_window()

#WAIT FOR COOKIES POPUP
accepted = 0
while accepted == 0:
    try:
        AcceptCookiesButton = browser.find_element(By.ID, 'onetrust-accept-btn-handler')
        AcceptCookiesButton.click()
        accepted = 1
        print("Cookies accepted!")
    except:
        print("Waiting for cookies...")
        time.sleep(2) 

#LOGING IN
BetfairEmailInput = browser.find_element(By.ID, 'ssc-liu')
BetfairEmailInput.clear()
BetfairEmailInput.send_keys(betfair_email)
BetfairPasswordInput = browser.find_element(By.ID, 'ssc-lipw')
BetfairPasswordInput.clear()
BetfairPasswordInput.send_keys(betfair_password)
BetfairLoginButton = browser.find_element(By.ID, 'ssc-lis')
BetfairLoginButton.click()
print("Successfully logged in!")

### Looping through all the games

In [ ]:
#SET FIRST PAGE
pages_to_loop = True
current_page = browser.current_url

#ENTER LOOP FOR EACH PAGE
while pages_to_loop:
    print(f'Buscando dados da página {current_page}')
    time.sleep(2)
    
    #ENTER LOOP FOR EACH LEAGUE IN PAGE
    leagues_left = True
    league_counter = 1
    while leagues_left:
        
        try:
            LigaElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[1]/div/span')
            print(f'Capturando {LigaElement.text}')
            
            #ENTER LOOP FOR EACH GAME IN LEAGUE
            games_left = True
            games_counter = 1
            while games_left:
                try:
                    DateElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/bf-livescores/section/div/div/data-bf-livescores-start-date/ng-include/div/div')
                    HomeTeamElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/ul[1]/li[1]')
                    AwayTeamElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/ul[1]/li[2]')
                    BackHomeElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[1]/button/label[1]')
                    LayHomeElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[2]/button/label[1]')
                    BackDrawElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[1]/button/label[1]')
                    LayDrawElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[2]/button/label[1]')
                    BackAwayElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[3]/ours-price-button[1]/button/label[1]')
                    LayAwayElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[3]/ours-price-button[2]/button/label[1]')
                    print(DateElement.text, ' ', HomeTeamElement.text, ' x ', AwayTeamElement.text)
                    print(f'Odds {BackHomeElement.text}/{LayHomeElement.text}/{BackDrawElement.text}/{LayDrawElement.text}/{BackAwayElement.text}/{LayAwayElement.text}')
                    games_counter += 1
                    time.sleep(2)
                    
                #EXIT GAME LOOP
                except:
                    print("Essa liga não tem mais jogos!")
                    games_left = False
                    time.sleep(2)
                
            league_counter += 1
            time.sleep(2)
            
        #EXIT LEAGUE LOOP
        except:
            print("Não há mais ligas nessa página!")
            leagues_left = False
            time.sleep(2)
    
    #MOVE TO NEXT PAGE
    previous_page = current_page
    NextButton = browser.find_element(By.XPATH, '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[4]/bf-coupon-page-navigation/ul/li[4]/a/label')
    NextButton.click()
    current_page = browser.current_url
    
    #IF LAST PAGE, END LOOP
    if previous_page == current_page:
        pages_to_loop = False
    time.sleep(2)

### Closing Edge Webdriver

In [ ]:
browser.close()
browser.quit()
print("Browser closed!")